In [1]:
import sys
import os
import pandas as pd
import string
import numpy as np
import re
import spacy
import es_core_news_sm
from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data import BucketIterator
from sklearn.model_selection import train_test_split
import torch
from model.Utils import GradientDescentMomentum
from model.Utils import exponentialDecaySchedule
from model.Seq2Seq_rnn import Seq2Seq_rnn

In [2]:
data_path = os.path.join(sys.path[0], 'data/spa.txt')
lines= pd.read_table(data_path,  names =['eng', 'spa', 'comments'])
train,valid = train_test_split(lines, test_size=0.1)

In [3]:
train.to_csv('train.csv')
valid.to_csv('valid.csv')

In [4]:
# use spacy for tokenization - provides good support for tokenization in languages other than english
eng_field = Field(tokenize="spacy",
                 tokenizer_language="en",
                 init_token="<sos>",
                 eos_token = "<eos>",
                 lower = True)

spa_field = Field(tokenize="spacy",
                 tokenizer_language="es",
                 init_token="<sos>",
                 eos_token = "<eos>",
                 lower=True)

/Users/khaira/Documents/NeuralMachineTranslation/venv/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [5]:
tabular_data_fields = [("id", None), ('eng', eng_field), ('spa', spa_field),('comments',None)]

In [6]:
train, valid = TabularDataset.splits(

path = './',
train='train.csv',
validation='valid.csv',
format = 'csv',
skip_header = True,
fields = tabular_data_fields)

/Users/khaira/Documents/NeuralMachineTranslation/venv/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/Users/khaira/Documents/NeuralMachineTranslation/venv/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [7]:
eng_field.build_vocab(train, valid)

In [8]:
spa_field.build_vocab(train, valid)

In [15]:
train_iter, val_iter = BucketIterator.splits((train, valid),
batch_size = 32, sort_key = lambda x: len(x.eng))

/Users/khaira/Documents/NeuralMachineTranslation/venv/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [18]:
def confirm_mapping(src, trg):
    src = np.copy(src)
    vfunc = np.vectorize(lambda x: eng_field.vocab.itos[x])
    saved_src = vfunc(src)
    
    trg = np.copy(trg)
    xfunc = np.vectorize(lambda x: spa_field.vocab.itos[x])
    saved_trg = xfunc(trg)
    
    return saved_src, saved_trg

In [19]:
spa = "spa"
eng= "eng"

In [20]:
for i, batch in enumerate(val_iter):
    print(type(batch.eng))
    saved = batch.spa.numpy()
    if (type(batch.eng) == torch.Tensor):
        src2 = batch.eng[:,0].numpy()
        trg2 = batch.spa[:,0].numpy()
    print(batch.eng.shape)
    print(batch.spa.shape)
    break

<class 'torch.Tensor'>
torch.Size([5, 32])
torch.Size([9, 32])


In [43]:
seq2seq_obj = Seq2Seq_rnn(eos_int=spa_field.vocab.stoi['<eos>'], sos_int=spa_field.vocab.stoi['<sos>']
                          , vocab_size_src = len(eng_field.vocab), vocab_size_trg=len(spa_field.vocab),
                          dim_embed_src=812, dim_embed_trg =812, src_map_i2c = eng_field.vocab.itos,
                          trg_map_i2c = spa_field.vocab.itos, num_neurons_encoder = 812, num_neurons_decoder = 812,
                          optim = GradientDescentMomentum)

In [44]:
eng = "eng"
spa = "spa"
lr_schedule = exponentialDecaySchedule(0.96, 100)

In [ ]:
# Works fine but have to train for a really long time 
seq2seq_obj.train( num_epochs=5000, valid_loader=val_iter, data_loader = train_iter, batch_size = 32, src_name = eng, trg_name=spa, 
                  padding_idx=spa_field.vocab.stoi['<pad>'], learning_schedule=lr_schedule, _testing=5, learn_rate=0.001)


  0%|          | 0/5000 [00:00<?, ?it/s]

Entering epoch: 0, batch number 0!
Entering epoch: 0, batch number 1!
Entering epoch: 0, batch number 2!
Entering epoch: 0, batch number 3!
Entering epoch: 0, batch number 4!
old learn rate: 0.001, new learn rate: 0.001
?cometiendo
Batch 0, input_sentence: <sos> we understand . <eos> translated sentence: ?cometiendo


  0%|          | 1/5000 [00:28<39:42:01, 28.59s/it]

Epoch num: 0, Train Loss: 157.7703411484086, Validation Loss: 64.26101935860306
Epoch num: 0, Train Loss: 157.7703411484086
Entering epoch: 1, batch number 0!
Entering epoch: 1, batch number 1!
Entering epoch: 1, batch number 2!
Entering epoch: 1, batch number 3!
Entering epoch: 1, batch number 4!
old learn rate: 0.001, new learn rate: 0.0009995918633652226
<pad>
Batch 0, input_sentence: <sos> we understand . <eos> translated sentence: <pad>


  0%|          | 2/5000 [00:58<40:20:36, 29.06s/it]

...

Epoch num: 32, Train Loss: nan, Validation Loss: nan
Epoch num: 32, Train Loss: nan
Entering epoch: 33, batch number 0!
Entering epoch: 33, batch number 1!
Entering epoch: 33, batch number 2!
Entering epoch: 33, batch number 3!
Entering epoch: 33, batch number 4!
old learn rate: 0.0008061059197183605, new learn rate: 0.0007953194753543122
cansar
Batch 0, input_sentence: <sos> we understand . <eos> translated sentence: cansar


  1%|          | 34/5000 [28:37<86:51:46, 62.97s/it]

Epoch num: 33, Train Loss: nan, Validation Loss: nan
Epoch num: 33, Train Loss: nan
Entering epoch: 34, batch number 0!
Entering epoch: 34, batch number 1!
Entering epoch: 34, batch number 2!
Entering epoch: 34, batch number 3!
Entering epoch: 34, batch number 4!
old learn rate: 0.0007953194753543122, new learn rate: 0.0007843571080349531
cansar
Batch 0, input_sentence: <sos> we understand . <eos> translated sentence: cansar


  1%|          | 35/5000 [29:52<92:00:58, 66.72s/it]

Epoch num: 34, Train Loss: nan, Validation Loss: nan
Epoch num: 34, Train Loss: nan
Entering epoch: 35, batch number 0!
Entering epoch: 35, batch number 1!
Entering epoch: 35, batch number 2!
Entering epoch: 35, batch number 3!
